Установка необходимых системных библиотек (кодеков для работы с изображением)

In [4]:
#!sudo apt-get update && sudo apt-get install ffmpeg libsm6 libxext6 -y

Установка необходимых библиотек

In [5]:
#%pip install jupyterlab ultralytics torch cvzone pandas openpyxl

In [4]:
import torch
from ultralytics import YOLO

# Загрузка первой модели
model1 = YOLO(r'best_1.pt')
if model1.model:
    model1 = model1.model.to('cpu').fuse()
else:
    print("Ошибка при загрузке первой модели")

# Загрузка второй модели
model2 = YOLO(r'best_2.pt')
if model2.model:
    model2 = model2.model.to('cpu').fuse()
else:
    print("Ошибка при загрузке второй модели")

merged_model_states_file_name = 'merged_yolov8.pth'

# Проверка успешности загрузки обеих моделей
if model1 and model2:
    # Усреднение словарей состояния
    model1_state_dict = model1.state_dict()
    model2_state_dict = model2.state_dict()

    new_state_dict = {}
    for k in model1_state_dict.keys():
        new_state_dict[k] = (model1_state_dict[k] + model2_state_dict[k]) / 2

    # Сохранение усредненного словаря состояния
    torch.save({'model': new_state_dict}, merged_model_states_file_name)

    print("Словари состояния усреднены и успешно сохранены.")
else:
    print("Невозможно усреднить словари состояния из-за отсутствия моделей.")

Model summary (fused): 168 layers, 3006038 parameters, 0 gradients
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients
Словари состояния усреднены и успешно сохранены.


Теперь у нас есть контрольная точка: файл с сохранёнными весами. Сымитируем передачу этой информации в другое окружение и обучение новой модели.

Создаём третью модель из файла с сохранёнными усреднёнными состояниями.

Этот код загружает усредненный словарь состояния модели из файла merged_model_states_file_name и сохраняет его в переменную merged_model_state_dict.

In [13]:
states = torch.load(merged_model_states_file_name, map_location=torch.device('cpu'))
merged_model_state_dict = states['model']

Ключевой момент - состояния, сохранённые с помощью torch, не являются сериализованной YOLO моделью, в отличие от best_1.pt и best_2.pt. 

Поэтому для получения модели из усреднённой модели создаём копию модели 2, после чего подгружаем веса, экспортированные на предыдущем шаге.

In [15]:
model3 = YOLO(r'best_2.pt')
model3.fuse()
model3.model.load_state_dict(merged_model_state_dict)

# Проверка успешной загрузки модели
if model3.model:
    print("Модель успешно загружена.")
else:
    print("Ошибка при загрузке модели.")

Model summary (fused): 168 layers, 3006038 parameters, 0 gradients
Модель успешно загружена.


Теперь можно использовать model3 для распознавания. 

Ещё один нюанс - при использовании jupyter блокнота в datalore не будет работать вывод картинок на экран, поэтому добавим переменную have_display, которая будет True только в ситуации, когда запускаем код локально, без jupyter блокнота, а как python код.

In [16]:
have_display = False

In [1]:
import cv2
import pandas as pd
import time
import cvzone
from copy import deepcopy
from ultralytics import YOLO

classNames = ['crop', 'weed']  # Список классов объектов

# Загрузка исходного видео
cap = cv2.VideoCapture('pred_t.mp4')

# Загрузка модели из объединенного словаря состояния
model3 = YOLO(r'best_2.pt')

# Проверка успешного открытия видео
if not cap.isOpened():
    print("Ошибка при открытии видеофайла.")
    exit()

# Получение параметров видео
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Определение кодека и создание объекта VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.avi', fourcc, fps, (width, height))

# Создание DataFrame для хранения обнаруженных объектов
df = pd.DataFrame(columns=['Class', 'Confidence', 'X1', 'Y1', 'X2', 'Y2'])

# Инициализация переменных для расчета FPS
prev_frame_time = 0
new_frame_time = 0

# Определение переменной have_display
have_display = True

while True:
    new_frame_time = time.time()

    # Захват кадра
    success, img = cap.read()

    if not success:
        print("Ошибка при считывании кадра из видеопотока.")
        break

    # Обнаружение объектов
    results = model3(img)

    for r in results:
        for b in r.boxes:
            x1, y1, x2, y2 = int(b.xyxy[0][0]), int(b.xyxy[0][1]), int(b.xyxy[0][2]), int(b.xyxy[0][3])
            conf = float(b.conf)
            cls = int(b.cls)

            try:
                if 0 <= cls < len(classNames):
                    cvzone.cornerRect(img, (x1, y1, x2 - x1, y2 - y1))
                    cvzone.putTextRect(img, f'{classNames[cls]} {conf:.2f}', (max(0, x1), max(35, y1)), scale=1, thickness=1)
                else:
                    print(f"Предупреждение: Индекс класса {cls} вышел за пределы диапазона. Используется метка по умолчанию.")
                    cvzone.putTextRect(img, "Неизвестный класс", (max(0, x1), max(35, y1)), scale=1, thickness=1)

                df = pd.concat([df, pd.DataFrame({'Class': classNames[cls], 'Confidence': conf, 'X1': x1, 'Y1': y1, 'X2': x2, 'Y2': y2}, index=[0])], ignore_index=True)
            except IndexError:
                print("Ошибка: Индекс за пределами диапазона. Пропуск обнаружения.")

    fps = 1 / (new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time

    # Запись кадра в выходное видео
    out.write(img)

    if have_display:
        cv2.putText(img, f"FPS: {int(fps)}", (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Изображение", img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

df.to_excel(r'detections3.xlsx', index=False)

# Освобождение ресурсов VideoCapture и VideoWriter
cap.release()
out.release()

if have_display:
    cv2.destroyAllWindows()



0: 640x640 1 crop, 1229.0ms
Speed: 19.0ms preprocess, 1229.0ms inference, 2473.0ms postprocess per image at shape (1, 3, 640, 640)



C:\Users\Muntadher\AppData\Local\Temp\ipykernel_7528\1677540794.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({'Class': classNames[cls], 'Confidence': conf, 'X1': x1, 'Y1': y1, 'X2': x2, 'Y2': y2}, index=[0])], ignore_index=True)


0: 640x640 1 crop, 1140.0ms
Speed: 17.0ms preprocess, 1140.0ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 crop, 613.0ms
Speed: 12.0ms preprocess, 613.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 crop, 625.0ms
Speed: 12.0ms preprocess, 625.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 crop, 799.0ms
Speed: 21.0ms preprocess, 799.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 crop, 516.0ms
Speed: 33.0ms preprocess, 516.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 crop, 488.0ms
Speed: 32.0ms preprocess, 488.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 crop, 436.0ms
Speed: 21.0ms preprocess, 436.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 crop, 475.0ms
Speed: 10.0ms preprocess, 475.0ms inference, 1.0ms postprocess per image at shape 